# **Data Generator**

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import ta.momentum as momentum
from datetime import datetime, timedelta
import time
import plotly.graph_objects as go

def get_historical_data(symbol, timeframe, start_date, end_date):
    if not mt5.initialize():
        print("initialize() failed ☢️")
        mt5.shutdown()
        return None

    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    print(f"Retrieved {len(rates)} data points")

    if rates is None:
        print("Failed to retrieve historical data. ☢️")
        mt5.shutdown()
        return None

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    mt5.shutdown()

    return df


def calculate_rsi(df, period=14):
    try:
        rsi_indicator = momentum.RSIIndicator(df["close"], window=period)
        df["rsi"] = rsi_indicator.rsi()
    except Exception as e:
        print(f"Error calculating RSI: {e}")

def find_filling_mode(symbol):
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC
        }

        result = mt5.order_check(request)

        if result.comment == "Done":
            print("Trade was closed")
            break

    return i


## **Cumulated profit (BOOM 1000/500/300)**

In [2]:

def backtest_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(3, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]
        confirmation_bar = df.iloc[i - 2]

        # Check for sell conditions
        if current_bar["rsi"] > 70 and confirmation_bar["close"] > confirmation_bar["open"] \
                and previous_bar["close"] < previous_bar["open"] \
                and current_bar["close"] < current_bar["open"]:
            
            if not in_trade:
                # Open a sell trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "SELL"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            if current_bar["close"] < current_bar["open"]:
                # Close the trade
                trades[-1]["exit_time"] = current_bar.name
                trades[-1]["exit_price"] = current_bar["close"]
                trades[-1]["pnl"] = trades[-1]["entry_price"] - trades[-1]["exit_price"]
                trades[-1]["pnl"] *= lot_size
                in_trade = False

    return trades

if __name__ == "__main__":
    symbols = ["Boom 1000 Index", "Boom 500 Index", "Boom 300 Index"]
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 7.0

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 7, 16)
    end_date = datetime.now()

    total_trades = 0
    total_profit = 0

    for symbol in symbols:
        # Step 1: Load historical data within the specified date range
        df = get_historical_data(symbol, timeframe, start_date, end_date)

        # Step 2: Calculate RSI
        calculate_rsi(df, period)

        # Step 3: Backtest the strategy
        trades = backtest_strategy(df, lot_size)

        # Step 4: Print the backtest results
        total_pnl = 0
        for trade in trades:
            print(f"Symbol: {symbol}")
            print(f"Trade Type: {trade['trade_type']}")
            print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
            print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
            print(f"Profit/Loss: {trade['pnl']:.2f}\n")
            total_pnl += trade['pnl']

        print(f"Total Profit/Loss for {symbol}: {total_pnl:.2f}")
        print(f"Total Trades for {symbol}: {len(trades)}")

        # Update cumulative values
        total_trades += len(trades)
        total_profit += total_pnl

    # Print the total cumulative values across symbols
    print(f"Total Trades across symbols: {total_trades}")
    print(f"Total Cumulative Profit: {total_profit:.2f}")

Retrieved 45910 data points
Symbol: Boom 1000 Index
Trade Type: SELL
Entry Time: 2023-07-16 00:04:00, Entry Price: 10536.996
Exit Time: 2023-07-16 00:04:00, Exit Price: 10536.394
Profit/Loss: 0.60

Symbol: Boom 1000 Index
Trade Type: SELL
Entry Time: 2023-07-16 00:22:00, Entry Price: 10540.087
Exit Time: 2023-07-16 00:22:00, Exit Price: 10539.432
Profit/Loss: 0.65

Symbol: Boom 1000 Index
Trade Type: SELL
Entry Time: 2023-07-16 00:29:00, Entry Price: 10558.359
Exit Time: 2023-07-16 00:29:00, Exit Price: 10557.755
Profit/Loss: 0.60

Symbol: Boom 1000 Index
Trade Type: SELL
Entry Time: 2023-07-16 00:34:00, Entry Price: 10583.411
Exit Time: 2023-07-16 00:34:00, Exit Price: 10582.809
Profit/Loss: 0.60

Symbol: Boom 1000 Index
Trade Type: SELL
Entry Time: 2023-07-16 00:42:00, Entry Price: 10582.102
Exit Time: 2023-07-16 00:42:00, Exit Price: 10581.422
Profit/Loss: 0.68

Symbol: Boom 1000 Index
Trade Type: SELL
Entry Time: 2023-07-16 02:10:00, Entry Price: 10550.274
Exit Time: 2023-07-16 02:

## **Cumulated profit (CRASH 1000/500/300)**

In [3]:
def backtest_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(3, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]
        confirmation_bar = df.iloc[i - 2]

        # Check for buy conditions
        if current_bar["rsi"] < 30 and confirmation_bar["close"] < confirmation_bar["open"] \
                and previous_bar["close"] > previous_bar["open"] \
                and current_bar["close"] > current_bar["open"]:
            
            if not in_trade:
                # Open a buy trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "BUY"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            if current_bar["close"] > current_bar["open"]:
                # Close the trade
                trades[-1]["exit_time"] = current_bar.name
                trades[-1]["exit_price"] = current_bar["close"]
                trades[-1]["pnl"] = trades[-1]["exit_price"] - trades[-1]["entry_price"]
                trades[-1]["pnl"] *= lot_size
                in_trade = False

    return trades

if __name__ == "__main__":
    symbols = ["Crash 1000 Index", "Crash 500 Index", "Crash 300 Index"]
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 7.0

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 7, 16)
    end_date = datetime.now()

    total_trades = 0
    total_profit = 0

    for symbol in symbols:
        # Step 1: Load historical data within the specified date range
        df = get_historical_data(symbol, timeframe, start_date, end_date)

        # Step 2: Calculate RSI
        calculate_rsi(df, period)

        # Step 3: Backtest the strategy
        trades = backtest_strategy(df, lot_size)

        # Step 4: Print the backtest results
        total_pnl = 0
        for trade in trades:
            print(f"Symbol: {symbol}")
            print(f"Trade Type: {trade['trade_type']}")
            print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
            print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
            print(f"Profit/Loss: {trade['pnl']:.2f}\n")
            total_pnl += trade['pnl']

        print(f"Total Profit/Loss for {symbol}: {total_pnl:.2f}")
        print(f"Total Trades for {symbol}: {len(trades)}")

        # Update cumulative values
        total_trades += len(trades)
        total_profit += total_pnl

    # Print the total cumulative values across symbols
    print(f"Total Trades across symbols: {total_trades}")
    print(f"Total Cumulative Profit: {total_profit:.2f}")


Retrieved 45910 data points
Symbol: Crash 1000 Index
Trade Type: BUY
Entry Time: 2023-07-15 23:06:00, Entry Price: 7505.162
Exit Time: 2023-07-15 23:06:00, Exit Price: 7505.579
Profit/Loss: 0.42

Symbol: Crash 1000 Index
Trade Type: BUY
Entry Time: 2023-07-15 23:27:00, Entry Price: 7504.018
Exit Time: 2023-07-15 23:27:00, Exit Price: 7504.444
Profit/Loss: 0.43

Symbol: Crash 1000 Index
Trade Type: BUY
Entry Time: 2023-07-15 23:32:00, Entry Price: 7487.932
Exit Time: 2023-07-15 23:32:00, Exit Price: 7488.413
Profit/Loss: 0.48

Symbol: Crash 1000 Index
Trade Type: BUY
Entry Time: 2023-07-16 00:03:00, Entry Price: 7490.442
Exit Time: 2023-07-16 00:03:00, Exit Price: 7490.904
Profit/Loss: 0.46

Symbol: Crash 1000 Index
Trade Type: BUY
Entry Time: 2023-07-16 00:58:00, Entry Price: 7505.47
Exit Time: 2023-07-16 00:58:00, Exit Price: 7505.897
Profit/Loss: 0.43

Symbol: Crash 1000 Index
Trade Type: BUY
Entry Time: 2023-07-16 02:17:00, Entry Price: 7519.425
Exit Time: 2023-07-16 02:17:00, Exit 